In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile


ModuleNotFoundError: No module named 'seaborn'

In [2]:
%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

mkdir: ../data: File exists
--2023-04-30 23:34:16--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M   746KB/s    in 32s     

2023-04-30 23:34:49 (2.51 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [6]:
import os
import glob


def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}

    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}

        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []

            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)

            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(
                        1 if sentiment == 'pos' else 0)

            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                "{}/{} data size does not match labels size".format(
                    data_type, sentiment)

    return data, labels


In [7]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
    len(data['train']['pos']), len(data['train']['neg']),
    len(data['test']['pos']), len(data['test']['neg'])))


IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [8]:
from sklearn.utils import shuffle


def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""

    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']

    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)

    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test


In [9]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(
    len(train_X), len(test_X)))



IMDb reviews (combined): train = 25000, test = 25000


In [10]:
print(train_X[100])
print(train_y[100])


A great film this, and a shame that it will receive little attention outside of arthouse circles and students who stay up until two in the morning to watch it on Channel Four.<br /><br />The plot is a simple one but works very effectively, the blurring between child-like fantasy and hard-hitting nightmare is very well blurred. The budget looks pretty low, but to the credit of those involved it doesn't show too often. It also hasn't dated that much either.<br /><br />I was lucky enough to tape this off the telly when it was on a few years ago, and it has withstood half-a-dozen viewings. It's one of those films that won't appeal to all; though as usual, those with a more thoughtful approach to cinema would get a lot out of this.<br /><br />Charlotte Buerke puts in a good performance as Anna, the spoilt brat and it is a shame she seems to have gone from the acting scene. Cross is also very good, carrying the stature of his character very well within the context of the picture.<br /><br />

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup


def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()

    text = BeautifulSoup(review, "html.parser").get_text()  # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())  # Convert to lower case
    words = text.split()  # Split string into words
    words = [w for w in words if w not in stopwords.words(
        "english")]  # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words]  # stem

    return words


In [4]:
review_to_words(train_X[100])


NameError: name 'train_X' is not defined